## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current_Description
0,0,Georgetown,MY,2022-07-15 19:40:52,5.4112,100.3354,82.89,93,20,0.00,few clouds
1,1,Vila,PT,2022-07-15 19:42:47,42.0304,-8.1588,63.48,62,35,2.01,scattered clouds
2,2,Puerto Ayora,EC,2022-07-15 19:45:22,-0.7393,-90.3518,76.96,92,65,8.01,broken clouds
3,3,Ushuaia,AR,2022-07-15 19:47:46,-54.8000,-68.3000,35.26,60,20,6.91,few clouds
4,4,Hermanus,ZA,2022-07-15 19:45:23,-34.4187,19.2345,55.80,85,100,9.33,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
Desired_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
Desired_cities_df.head(10)

,Unnamed: 0,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current_Description
0,0,Georgetown,MY,2022-07-15 19:40:52,5.4112,100.3354,82.89,93,20,0.00,few clouds
2,2,Puerto Ayora,EC,2022-07-15 19:45:22,-0.7393,-90.3518,76.96,92,65,8.01,broken clouds
9,9,Constantine,DZ,2022-07-15 19:52:13,36.3650,6.6147,81.81,34,0,10.36,clear sky
19,19,Atuona,PF,2022-07-15 19:45:28,-9.8000,-139.0333,76.66,74,1,12.62,clear sky
20,20,Vaini,TO,2022-07-15 19:44:14,-21.2000,-175.2000,75.36,94,20,5.75,few clouds
22,22,Hithadhoo,MV,2022-07-15 19:45:29,-0.6000,73.0833,83.34,74,100,15.70,overcast clouds
24,24,Ternate,ID,2022-07-15 19:45:30,0.8000,127.4000,76.46,90,100,10.25,moderate rain
25,25,Ponta Do Sol,PT,2022-07-15 19:45:30,32.6667,-17.1000,76.01,79,15,6.06,few clouds
27,27,Jamestown,US,2022-07-15 19:45:31,42.0970,-79.2353,81.05,40,75,11.50,broken clouds
29,29,Kyabe,TD,2022-07-15 19:45:32,9.4515,18.9449,75.20,89,100,3.71,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
Desired_cities_df.count()
Desired_cities_df.isnull().sum()

Unnamed: 0             0
City                   0
Country                0
Date                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current_Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
Cleared_df= Desired_cities_df.dropna()
Cleared_df.isnull().sum()

Unnamed: 0             0
City                   0
Country                0
Date                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current_Description    0
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = Cleared_df[["City", "Country", "Max Temp", "Current_Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current_Description,Lat,Lng,Hotel Name
0,Georgetown,MY,82.89,few clouds,5.4112,100.3354,
2,Puerto Ayora,EC,76.96,broken clouds,-0.7393,-90.3518,
9,Constantine,DZ,81.81,clear sky,36.3650,6.6147,
19,Atuona,PF,76.66,clear sky,-9.8000,-139.0333,
20,Vaini,TO,75.36,few clouds,-21.2000,-175.2000,
22,Hithadhoo,MV,83.34,overcast clouds,-0.6000,73.0833,
24,Ternate,ID,76.46,moderate rain,0.8000,127.4000,
25,Ponta Do Sol,PT,76.01,few clouds,32.6667,-17.1000,
27,Jamestown,US,81.05,broken clouds,42.0970,-79.2353,
29,Kyabe,TD,75.20,overcast clouds,9.4515,18.9449,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
        # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    #print(hotels)

    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [19]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df= hotel_df.loc[hotel_df["Hotel Name"] != ""]

,City,Country,Max Temp,Current_Description,Lat,Lng,Hotel Name
0,Georgetown,MY,82.89,few clouds,5.4112,100.3354,Cititel Penang
2,Puerto Ayora,EC,76.96,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
9,Constantine,DZ,81.81,clear sky,36.3650,6.6147,Grand Hotel Cirta
19,Atuona,PF,76.66,clear sky,-9.8000,-139.0333,Villa Enata
20,Vaini,TO,75.36,few clouds,-21.2000,-175.2000,Keleti Beach Resort
...,...,...,...,...,...,...,...
687,Orodara,BF,76.03,light rain,10.9492,-4.9342,Bkf Orodara
688,Minervino Murge,IT,79.23,clear sky,41.0843,16.0784,B&B Le Mura
691,Havelock,US,83.05,few clouds,34.8791,-76.9013,Sherwood Motel
693,Gigmoto,PH,81.16,overcast clouds,13.7796,124.3900,Cj's Travellers Inn


In [20]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [23]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]


In [24]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.4)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))